<a href="https://colab.research.google.com/github/jakovitz-W/CreditFraudDetection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTE: Dataset is too large to be committed to github, download & move csv file into working directory

In [ ]:
%pip install matplotlib
%pip install pandas

In [ ]:
#data preperation
df = pd.DataFrame(pd.read_csv("creditcard.csv"))



Preprocessing/Feature Analysis

In [ ]:
#create dataframes
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(pd.read_csv("creditcard.csv"))

frauds = df.where(df["Class"] == 1)
frauds.dropna(inplace=True)
print(frauds.head())

legit = df.where(df["Class"] == 0)
legit.dropna(inplace=True)

legit = legit.sample(frac=0.005, replace=False, random_state=1)

In [ ]:
#plots
plt.scatter(frauds['Time'], frauds['V1'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V1'], color="blue", label='Legitimate')

plt.title("V1")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V2'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V2'], color="blue", label='Legitimate')

plt.title("V2")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V3'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V3'], color="blue", label='Legitimate')

plt.title("V3")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V4'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V4'], color="blue", label='Legitimate')

plt.title("V4")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V5'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V5'], color="blue", label='Legitimate')

plt.title("V5")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V6'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V6'], color="blue", label='Legitimate')

plt.title("V6")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V7'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V7'], color="blue", label='Legitimate')

plt.title("V7")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V8'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V8'], color="blue", label='Legitimate')

plt.title("V8")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V9'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V9'], color="blue", label='Legitimate')

plt.title("V9")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V10'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V10'], color="blue", label='Legitimate')

plt.title("V10")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V11'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V11'], color="blue", label='Legitimate')

plt.title("V11")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V12'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V12'], color="blue", label='Legitimate')

plt.title("V12")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V13'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V13'], color="blue", label='Legitimate')

plt.title("V13")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V14'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V14'], color="blue", label='Legitimate')

plt.title("V14")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V15'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V15'], color="blue", label='Legitimate')

plt.title("V15")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V16'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V16'], color="blue", label='Legitimate')

plt.title("V16")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V17'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V17'], color="blue", label='Legitimate')

plt.title("V17")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V18'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V18'], color="blue", label='Legitimate')

plt.title("V18")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V19'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V19'], color="blue", label='Legitimate')

plt.title("V19")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V20'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V20'], color="blue", label='Legitimate')

plt.title("V20")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V21'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V21'], color="blue", label='Legitimate')

plt.title("V21")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V22'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V22'], color="blue", label='Legitimate')

plt.title("V22")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V23'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V23'], color="blue", label='Legitimate')

plt.title("V23")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V24'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V24'], color="blue", label='Legitimate')

plt.title("V24")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V25'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V25'], color="blue", label='Legitimate')

plt.title("V25")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V26'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V26'], color="blue", label='Legitimate')

plt.title("V26")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()


plt.scatter(frauds['Time'], frauds['V27'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V27'], color="blue", label='Legitimate')

plt.title("V27")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['V28'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['V28'], color="blue", label='Legitimate')

plt.title("V28")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()

plt.scatter(frauds['Time'], frauds['Amount'], color="red", label='Fraudulent')
plt.scatter(legit['Time'], legit['Amount'], color="blue", label='Legitimate')

plt.title("Amount")
plt.xlabel('Time Elapsed From First Transaction')
plt.legend()
plt.show()